In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
import numpy as np
import os
import shutil
import random

In [2]:
torch.cuda.set_device(0)
device = torch.device("cuda")

def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_all_seeds(42)

# Oversampling

In [3]:
from tqdm import tqdm
def check_classes_amount(set_with_labels):
    counter = [1,0,0,0,0,0,0,0,0,0,
               0,0,0,0,0,0,0,0,0,0,
               0,0,0,0,0,0,0,0,0,0,
               0,0,0,0,0,0,0,0,0,0,
               0,0,0,0,0,0,0,0,0,0]
    count = 0
    for j in tqdm(range(88010)):
        if set_with_labels[j][1] == set_with_labels[j+1][1]:
            counter[count] = counter[count] + 1
        else:
            count += 1
            counter[count] = counter[count] + 1

    return counter


In [4]:
def over_sample_bread_carbon():
    classes_amounts = check_classes_amount(trainset)
    sub_folders = os.listdir("./train")
    list_of_images_bread = os.listdir(f"./train/{sub_folders[10]}")
    list_of_images_carbon = os.listdir(f"./train/{sub_folders[13]}")

    to_oversample_bread = random.sample(list_of_images_bread, k=(1800 - classes_amounts[10]))
    to_oversample_carbon = random.sample(list_of_images_carbon, k= (1800 - 3 * classes_amounts[13]))

    # oversample bread class
    for i in range(len(to_oversample_bread)):
        shutil.copy(f"./train/{sub_folders[10]}/{to_oversample_bread[i]}", f"./train/{sub_folders[10]}/{[name[:-5] + '_1' + name[-5:] for name in to_oversample_bread][i]}")

    # oversample carbon class
    for i in range(classes_amounts[13]):
        shutil.copy(f"./train/{sub_folders[13]}/{list_of_images_carbon[i]}", f"./train/{sub_folders[13]}/{[name[:-5] + '_1' + name[-5:] for name in list_of_images_carbon][i]}")

    for i in range(classes_amounts[13]):
        shutil.copy(f"./train/{sub_folders[13]}/{list_of_images_carbon[i]}", f"./train/{sub_folders[13]}/{[name[:-5] + '_2' + name[-5:] for name in list_of_images_carbon][i]}")

    for i in range(len(to_oversample_carbon)):
        shutil.copy(f"./train/{sub_folders[13]}/{to_oversample_carbon[i]}", f"./train/{sub_folders[13]}/{[name[:-5] + '_3' + name[-5:] for name in to_oversample_carbon][i]}")

In [5]:
trainset = datasets.ImageFolder("train/")
# over_sample_bread_carbon()

# Create val folder

In [6]:
def create_val_folder():
    sub_folders = os.listdir("./train")
    os.mkdir("./val")
    for sub_folder in sub_folders:
        os.mkdir(f"./val/{sub_folder}")

    for sub_folder in sub_folders:
        to_move = random.sample(os.listdir(f"./train/{sub_folder}"), k=360)
        for image in to_move:
            shutil.move(f"./train/{sub_folder}/{image}", f"./val/{sub_folder}/{image}")

# create_val_folder()

# Data preparation

In [7]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(0.5),
     transforms.RandomVerticalFlip(0.5),
     transforms.RandomRotation(24),
     transforms.ToTensor(),
     transforms.RandomErasing(0.4),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 64

In [8]:
trainset = datasets.ImageFolder("train/", transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valset = datasets.ImageFolder("val/", transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

testset = datasets.ImageFolder("test_all/", transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

classes = tuple(trainset.classes)

In [12]:
def get_accuracy(loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images = images.to(device)
            # calculate outputs by running images through the network
            outputs = net(images).cpu()
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    return 100 * correct / total


# Net preparation

In [14]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
##Warstwakonwolucyjna
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3,stride=1,padding=0)
##Warstwamaxpooling
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(64,150,3)

        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(150, 150, 3)
        self.pool3 = nn.MaxPool2d(2)
        self.drop = nn.Dropout(p=0.2)
        self.conv4 = nn.Conv2d(150, 150, 3)
        self.pool4 = nn.MaxPool2d(2)

        self.fc0 = nn.Linear(600,300)
        # self.fc1 = nn.Linear(2200,1100)
        # self.fc2 = nn.Linear(1100,500)
        self.fc3 = nn.Linear(300, 150)
        self.fc4 = nn.Linear(150,50)


    def forward(self,x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.drop(x)
        x = self.pool4(F.relu(self.conv4(x)))
        x = torch.flatten(x,1)#flattenalldimensionsexceptbatch
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


net = Net().to(device)
net

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 150, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(150, 150, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout(p=0.2, inplace=False)
  (conv4): Conv2d(150, 150, kernel_size=(3, 3), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc0): Linear(in_features=600, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=150, bias=True)
  (fc4): Linear(in_features=150, out_features=50, bias=True)
)

In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [26]:
from tqdm import tqdm

epochs = 8
for epoch in range(epochs):

    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('[%d/%d] loss: %.3f' %
          (epoch+1, epochs,  running_loss / len(trainloader)))
    running_loss = 0.0


    print(f"accuracy on trainloader: {get_accuracy(trainloader)}")
    print(f"accuracy on valloader: {get_accuracy(valloader)}")
print('Finished Training')

1125it [01:39, 11.28it/s]


[1/8] loss: 1.579
accuracy on trainloader: 56.727777777777774
accuracy on valloader: 46.5


1125it [03:20,  5.61it/s]


[2/8] loss: 1.557
accuracy on trainloader: 57.43194444444445
accuracy on valloader: 46.49444444444445


1125it [02:34,  7.29it/s]


...

[6/8] loss: 1.473
accuracy on trainloader: 59.325
accuracy on valloader: 46.44444444444444


1125it [01:44, 10.74it/s]


[7/8] loss: 1.449
accuracy on trainloader: 59.50555555555555
accuracy on valloader: 46.272222222222226


1125it [01:43, 10.87it/s]


[8/8] loss: 1.435
accuracy on trainloader: 59.74444444444445
accuracy on valloader: 46.516666666666666
Finished Training


In [27]:
dataiter = iter(testloader)
images, labels = next(dataiter)

In [28]:
outputs = net(images.to(device))
#outputs
print(len(outputs))

64


In [29]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(len(predicted))))

Predicted:  elephant motor bacteria spice tomato   egg towel antenna spoon birch   tea acoustic monkey hammer flower kangaroo tomato acoustic  crab  frog bacteria  frog camera camera camera beetle  fish snake turtle   saw icecream motor fungus turtle truck   cat motor printer   tea hammer crocodilian monkey squash pizza echinoderm   cat bread   cat bridge elephant  corn  palm squash   tea spice beetle bicycle spider fungus swine  corn  bird fungus   cat


## Making predictions to .csv

In [30]:
correct = 0
total = 0
predictions = []

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        # calculate outputs by running images through the network
        outputs = net(images).cpu()
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)

        # print(predicted)
        for i in predicted:
            predictions.append(i.item())

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))

In [31]:
list_of_test_img = os.listdir(f"./test_all/test_all")
img_name = pd.DataFrame(list_of_test_img)
# print(list_of_test_img)
pred = pd.DataFrame(predictions)
print(pred.head())
results = pd.concat([img_name, pred], axis=1)

    0
0  47
1  31
2   2
3  14
4  45


In [32]:
print(len(img_name[0].unique()))
print(len(pred[0].unique()))

10000
50


In [33]:
results.to_csv('Kusmierz_Podolec_48.csv', header=None, index=None)